In [10]:
### AmirHoseinRostami
### 96101635
### VisionFinalProjectReport

# SimpleShow

In [ ]:
# just show
import numpy as np
import cv2
from datetime import datetime 

cap = cv2.VideoCapture('Video1.avi')

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if(frame is None):
        break
        
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('gray',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


# BackGroundExtration

In [11]:
# getting background
import numpy as np
import cv2
from skimage import data, filters

cap = cv2.VideoCapture('Video1.avi')
# Randomly select 25 frames 
frames_taken = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=25)
# Store selected frames in an array
frames = []
for fid in frames_taken:
    cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = cap.read()
    frames.append(frame)


background = np.median(frames, axis=0).astype(dtype=np.uint8)   
# Display median frame

cv2.imshow('background', background)
cv2.waitKey(0)
cv2.destroyAllWindows()

# MotionDetection

In [12]:

# Python program to implement  
# Motion Detector 
  
# importing OpenCV
import cv2
# importing datetime class from datetime library 
from datetime import datetime 

# Capturing video 
Video = cv2.VideoCapture('Video1.avi')
noOfFrames = int(Video.get(cv2.CAP_PROP_FRAME_COUNT))
frames = []
start= True;

#  while loop to treat stack of image as video 
while(Video.isOpened()): 
    
    # Reading frame(image) from video 
    check, frame = Video.read()
    
    if(start):
        startTime = datetime.now()
        start = False
        
    if(frame is None):
        endTime = datetime.now()
        break
        
    # Converting color image to gray_scale image 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    # Converting gray scale image to GaussianBlur  
    # so that change can be find easily 
    gray = cv2.GaussianBlur(gray, (21, 21), 0) 
  
    # Converting gray scale backGround to GaussianBlur  
    # so that change can be find easily 
    grayBackground = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY) 
    static_back = cv2.GaussianBlur(grayBackground, (21, 21), 0) 
    
    
    # Difference between static background  
    # and current frame(which is GaussianBlur) 
    diff_frame = cv2.absdiff(static_back, gray) 
  
    # If change in between static background and 
    # current frame is greater than 30 it will show white color(255) 
    thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1] 
    thresh_frame = cv2.dilate(thresh_frame, None, iterations = 2) 
  
    # Finding contour of moving object 
    cnts,_ = cv2.findContours(thresh_frame.copy(),  
                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
  

    t = datetime.now()
    
    contsInOneFrame = []
    for contour in cnts:
    
        if cv2.contourArea(contour) < 500: 
            continue

        (x, y, w, h) = cv2.boundingRect(contour) 
        data = (x, y, w, h)
        miniImage = frame[y:y+h,x:x+w]
        contsInOneFrame.append((miniImage,data,t))
        
        # making green rectangle arround the moving object 
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1) 


    frames.append(contsInOneFrame)
    
    # draw the text and timestamp on the frame
    text = "Contoured"
    cv2.putText(frame, "Status: {}".format(text), (10, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    #cv2.putText(frame, datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
    #    (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
  
    # Displaying image in gray_scale 
    cv2.imshow("Gray Frame", gray) 
  
    # Displaying the difference in currentframe to 
    # the staticframe(very first_frame) 
    cv2.imshow("Difference Frame", diff_frame) 
  
    # Displaying the black and white image in which if 
    # intensity difference greater than 30 it will appear white 
    cv2.imshow("Threshold Frame", thresh_frame) 
  
    # Displaying color frame with contour of motion of object 
    cv2.imshow("Color Frame", frame) 
  
    key = cv2.waitKey(20) 
    # if q entered whole process will stop 
    if key == ord('q'): 
        break

Video.release() 
# Destroying all the windows 
cv2.destroyAllWindows() 

# SomeNeeded Functions :)

In [13]:
from skimage import measure
import matplotlib.pyplot as plt
import numpy as np
import cv2

def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

def compare_images(imageA, imageB):
    # compute the mean squared error and structural similarity
    # index for the images
    imageA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    imageB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)

    m = mse(imageA, imageB)
    s = measure.compare_ssim(imageA, imageB)
    # setup the figure
    
    # mse & ssim
    return (m,s)

def sameSizing(pack1,pack2):
    first = pack1[0]
    second = pack2[0]
    xf,yf,_ = first.shape
    xs,ys,_ = second.shape
    areaF = xf*yf
    areaS = xs*ys
    
    if(areaF > areaS):
        return (first,cv2.resize(second,(yf,xf)))
    else:
        return (cv2.resize(first,(ys,xs)),second)


def findCenterDistance(pack1,pack2):
    import math
    #each pack is a (image,data,t)
    data1 = pack1[1]
    data2 = pack2[1]
    # data : x,y,w,h
    # center : x+w/2,y+h/2
    center1x = int((data1[0]+data1[2])/2)
    center1y = int((data1[1]+data1[3])/2)
    
    center2x = int((data2[0]+data2[2])/2)
    center2y = int((data2[1]+data2[3])/2)
    
    dist = math.sqrt(pow((center1x-center2x),2) +
                     pow((center1y-center2y),2))
    return dist
   
# def show(string,listImage):
#   for pack in listImage:
#        cv2.imshow(string,pack[0])
#        cv2.waitKey(0)
#        cv2.destroyAllWindows()

# Extracting Tubes

In [14]:
tubes = []
currentFrame = []
# each contor is a Image,Data,t
for i in range(len(frames[0])):
    
    # tube contains (pack,index) that pack is a (image,data,t)
    currentFrame.append((frames[0][i],i))
    tubes.append([frames[0][i]])
    #cv2.imshow("1",[frames[0][i][0])
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    
for i in range(1,len(frames)-1):
    beforeFrame = currentFrame
    currentFrame = frames[i+1]
    
    #show("current" + str(i),currentFrame)
    
    signFrames = []
    for currentMini in currentFrame:
        
        addToTube = True 
        # beforeMini is a (pack,index) -> pack is a (image,data,t)
        for beforeMini in beforeFrame:
            
            # findCenterDistance gets pack
            dist = findCenterDistance(beforeMini[0],currentMini)
            print("dist: " + str(dist))
            # gonna set
            distThreshold = 200
            if(dist > distThreshold):
                print("distIgnorance")
                continue
                
            #corr
            # sameSizing gets pack
            befo,curr = sameSizing(beforeMini[0],currentMini)
            MSE,SSIM = compare_images(befo,curr)
            print("MSE: " + str(MSE))
            mseErrorThreshold = 1000
            ssimErrorThreshold = 0.7
            #MSE < mseErrorThreshold and
            if(SSIM > ssimErrorThreshold):
                print("detected")
                # add to tubes
                index = beforeMini[1]
                tubes[index].append(currentMini)
                # signing
                signFrames.append((currentMini,index))
                addToTube = False
                break
        
        # add to tubes
        if(addToTube):
            tubes.append([currentMini])
            signFrames.append((currentMini,len(tubes)-1))
            print(len(tubes))
                
    currentFrame = signFrames        
                
print(len(tubes))          

def getHardCopy():
    hardCopy = []
    for tube in tubes:
        hardCopy.append(tube.copy())
    return hardCopy
        

# get a hard Copy 
mainTubes = getHardCopy()

dist: 24.331050121192877
MSE: 7546.213530655391
dist: 1.4142135623730951
MSE: 137.5284090909091
detected
dist: 0.0
MSE: 118.21670190274841
detected
dist: 1.0
MSE: 355.91472868217056
detected
dist: 0.0
MSE: 0.1564625850340136
detected
dist: 1.4142135623730951
MSE: 257.9829931972789
detected
dist: 0.0
MSE: 225.84580498866214
detected
dist: 1.0
MSE: 305.50267697798927
detected
dist: 1.0
MSE: 164.10767400356931
detected
dist: 1.0
MSE: 162.656875
detected
dist: 0.0
MSE: 186.366875
detected
dist: 1.0
MSE: 261.0429487179487
detected
dist: 0.0
MSE: 170.4766600920447
detected
dist: 1.4142135623730951
MSE: 396.439842209073
detected
dist: 0.0
MSE: 232.7034845496384
detected
dist: 0.0
MSE: 389.253036437247
detected
dist: 1.0
MSE: 238.56925207756234
detected
dist: 1.0
MSE: 204.48268698060943
detected
dist: 1.0
MSE: 140.21052631578948
detected
dist: 0.0
MSE: 337.76173541963016
detected
dist: 0.0
MSE: 329.624466571835
detected
dist: 1.0
MSE: 250.64207450693937
detected
dist: 1.0
MSE: 173.276844411979

MSE: 688.0016025641025
detected
dist: 5.385164807134504
MSE: 134.56661941885187
detected
dist: 189.76301009416983
MSE: 6158.179052343829
dist: 0.0
MSE: 74.2234375
detected
dist: 3.605551275463989
MSE: 398.4036568213783
detected
dist: 184.3827540742355
MSE: 6186.142864870713
dist: 0.0
MSE: 155.19375
detected
dist: 5.385164807134504
MSE: 215.65299523485365
detected
dist: 180.89223311131963
MSE: 6340.729748127978
dist: 0.0
MSE: 265.84296875
detected
dist: 3.605551275463989
MSE: 230.89467319668663
detected
dist: 175.5135322418189
MSE: 6255.361811475906
dist: 0.0
MSE: 410.12860310421286
detected
dist: 3.605551275463989
MSE: 111.29893651305188
detected
dist: 172.02906731131225
MSE: 6322.485723493393
dist: 0.0
MSE: 324.69253510716925
detected
dist: 3.1622776601683795
MSE: 162.66855791962175
detected
dist: 169.49926253526885
MSE: 6149.583451536643
dist: 1.0
MSE: 231.13795731707316
detected
dist: 4.47213595499958
MSE: 104.28920700162993
detected
dist: 166.02710622064097
MSE: 6161.978456523279
d

MSE: 6589.913336543091
dist: 103.0
MSE: 2638.3998015873017
dist: 0.0
MSE: 424.359126984127
detected
dist: 1.4142135623730951
MSE: 85.72825024437928
detected
dist: 16.278820596099706
MSE: 6532.042033235582
dist: 1.0
MSE: 25.51924528301887
detected
dist: 106.06601717798213
MSE: 6679.127565982405
dist: 103.00485425454472
MSE: 2796.114994934144
dist: 1.0
MSE: 190.73511904761904
detected
dist: 2.0
MSE: 217.07890818858561
detected
dist: 16.278820596099706
MSE: 6586.347890818859
dist: 1.4142135623730951
MSE: 29.102124183006534
detected
dist: 104.80935072788114
MSE: 6791.792635658915
dist: 102.0196059588548
MSE: 2721.53003875969
dist: 1.0
MSE: 236.02228682170542
detected
dist: 1.0
MSE: 218.3126984126984
detected
dist: 16.1245154965971
MSE: 6707.14126984127
dist: 1.0
MSE: 29.151304347826088
detected
dist: 102.82509421342633
MSE: 6737.880873279544
dist: 101.04454463255303
MSE: 2725.5277645942097
dist: 0.0
MSE: 170.14617940199335
detected
dist: 1.0
MSE: 86.07096774193549
detected
dist: 15.2970585

MSE: 325.6238705738706
detected
dist: 43.0
MSE: 8499.408669108669
dist: 1.0
MSE: 5237.513956507627
dist: 42.0
MSE: 5681.48182408309
dist: 178.35358140502814
MSE: 6678.990993184031
dist: 251.18120948828954
distIgnorance
64
dist: 205.7790076757102
distIgnorance
dist: 177.78920102188434
MSE: 3294.44419306184
dist: 158.39823231336896
MSE: 5445.7364208118925
dist: 0.0
MSE: 117.26794871794871
detected
dist: 269.8332818612263
distIgnorance
dist: 251.78165143631892
distIgnorance
dist: 241.04978738841484
distIgnorance
dist: 91.2414379544733
MSE: 6636.489865689866
dist: 1.0
MSE: 217.21587301587303
detected
dist: 12.806248474865697
MSE: 261.99428770463254
detected
dist: 35.73513677041127
MSE: 8554.309439219784
dist: 0.0
MSE: 3296.38285634119
dist: 178.35358140502814
MSE: 5700.1632996633
dist: 252.12100269513448
distIgnorance
65
dist: 192.33824372703418
MSE: 5586.903239289446
dist: 179.17868176767013
MSE: 6856.438575137942
dist: 1.0
MSE: 206.53047091412742
detected
dist: 255.41534801182172
distIgn

MSE: 1655.2081965290806
dist: 255.0019607767752
distIgnorance
94
dist: 277.41665415039523
distIgnorance
dist: 139.51702405083043
MSE: 9049.813112745098
dist: 256.1640099623677
distIgnorance
dist: 2.23606797749979
MSE: 286.63982843137256
detected
dist: 0.0
MSE: 6636.691402714932
dist: 155.72411502397438
MSE: 7656.419570135747
dist: 73.0
MSE: 11742.651892725762
dist: 277.41665415039523
distIgnorance
95
dist: 158.54967675779096
MSE: 7654.859152513295
dist: 2.8284271247461903
MSE: 133.3405315614618
detected
dist: 76.0
MSE: 12502.314768951977
dist: 118.6001686339442
MSE: 10105.652703110844
dist: 3.0
MSE: 1723.874599501602
dist: 255.15485494107298
distIgnorance
96
dist: 277.0
distIgnorance
dist: 137.3644786689776
MSE: 8895.455782312925
dist: 255.47602627252522
distIgnorance
dist: 1.0
MSE: 261.8888106966925
detected
dist: 0.0
MSE: 7574.245674740484
dist: 158.54967675779096
MSE: 8715.371725160652
dist: 76.0
MSE: 10940.96704922984
dist: 277.0
distIgnorance
97
dist: 161.37533888422976
MSE: 7604.

...

MSE: 506.7931392931393
detected
dist: 3.605551275463989
MSE: 136.4516339869281
detected
dist: 37.21558813185679
MSE: 4158.383786848072
dist: 2.23606797749979
MSE: 344.0399610136452
detected
dist: 102.17631819555841
MSE: 5675.874433106576
dist: 78.89233169326408
MSE: 4583.830041580041
542
dist: 5.0
MSE: 116.62458977965306
detected
dist: 38.118237105091836
MSE: 4199.453464052288
dist: 2.23606797749979
MSE: 307.88553113553115
detected
dist: 105.75916035975324
MSE: 5896.089411764706
dist: 81.12336284942828
MSE: 4668.148148148148
dist: 0.0
MSE: 108.19607843137256
detected
dist: 5.0
MSE: 210.6872848537005
detected
dist: 41.048751503547585
MSE: 4115.144983591186
dist: 2.23606797749979
MSE: 340.8209626630679
detected
dist: 110.75197515168747
MSE: 5880.537037037037
dist: 83.00602387778854
MSE: 4286.266483516483
dist: 0.0
MSE: 24.149859943977592
detected
dist: 5.0
MSE: 218.20437524635395
detected
dist: 44.01136216933077
MSE: 4160.761080034424
dist: 2.23606797749979
MSE: 214.67587719298245
detect

MSE: 230.75835275835277
detected
dist: 174.7483905505284
MSE: 4533.289166529673
dist: 30.479501308256342
MSE: 3504.185703185703
dist: 0.0
MSE: 338.7764227642276
detected
dist: 7.0710678118654755
MSE: 846.9680392156863
dist: 155.9134375222354
MSE: 7305.506753246753
dist: 186.3866948041088
MSE: 3400.0233766233764
557
dist: 148.86235252742716
MSE: 7506.610588235294
dist: 1.0
MSE: 144.58161764705883
detected
dist: 180.00555546982432
MSE: 3813.9635294117647
dist: 31.89043743820395
MSE: 3575.056721056721
dist: 0.0
MSE: 262.0178861788618
detected
dist: 6.0
MSE: 922.9215584415584
dist: 160.65490966665163
MSE: 5978.724778046812
dist: 191.43667360252581
MSE: 2487.1444713478613
558
dist: 154.5477272560163
MSE: 7283.73974025974
dist: 1.0
MSE: 277.8271162123386
detected
dist: 185.84133017173548
MSE: 3074.1628571428573
dist: 31.89043743820395
MSE: 3819.5264705882355
dist: 1.0
MSE: 256.3365079365079
detected
dist: 158.42032697857937
MSE: 6265.138014527845
dist: 1.4142135623730951
MSE: 204.52312925170

MSE: 80.41821946169772
detected
dist: 0.0
MSE: 225.29340431824903
detected
dist: 1.0
MSE: 158.57688991089393
detected
dist: 1.0
MSE: 98.59436619718309
detected
dist: 0.0
MSE: 203.48301369863015
detected
dist: 1.0
MSE: 93.5409615901155
detected
dist: 1.0
MSE: 136.21628029504743
detected
dist: 1.0
MSE: 113.53977562468128
detected
dist: 0.0
MSE: 62.283138401559455
detected
dist: 1.4142135623730951
MSE: 107.81722488038278
detected
dist: 0.0
MSE: 257.9445221445221
detected
dist: 0.0
MSE: 277.88607594936707
detected
dist: 1.4142135623730951
MSE: 96.90302893309223
detected
dist: 0.0
MSE: 65.09941520467837
detected
dist: 1.0
MSE: 232.26572528883185
detected
dist: 1.0
MSE: 62.72600747818861
detected
dist: 1.0
MSE: 64.19774011299435
detected
dist: 1.0
MSE: 78.50823529411765
detected
dist: 1.0
MSE: 248.1360465116279
detected
dist: 1.0
MSE: 58.78142076502732
detected
dist: 1.0
MSE: 95.81272401433692
detected
dist: 0.0
MSE: 199.42733073378236
detected
dist: 1.4142135623730951
MSE: 77.58160800552105

MSE: 3227.1037691001698
dist: 44.04543109109048
MSE: 4517.485881737311
dist: 79.47955712005447
MSE: 3278.777113752122
dist: 108.67382389517726
MSE: 4385.602920203735
596
dist: 346.0057802985378
distIgnorance
dist: 28.319604517012593
MSE: 466.95954734651406
dist: 123.08127396155761
MSE: 4793.906603872319
dist: 4.0
MSE: 70.26609492481202
detected
dist: 273.408119850161
distIgnorance
dist: 110.86027241532469
MSE: 3212.505183524797
dist: 147.6787053031005
MSE: 6277.978189429618
dist: 81.04319835741924
MSE: 3513.3130874060153
dist: 1.0
MSE: 110.7341935483871
detected
dist: 10.0
MSE: 2145.9129225205174
dist: 367.76351096866586
distIgnorance
dist: 99.18165152889924
MSE: 4258.541039339485
dist: 74.4043009509531
MSE: 5662.150730050934
dist: 121.0495766204905
MSE: 4986.547606659729
dist: 177.4288589829738
MSE: 6387.371539582322
597
dist: 358.36573496917924
distIgnorance
dist: 4.0
MSE: 4825.012397225077
dist: 362.465170740583
distIgnorance
dist: 305.04098085339285
distIgnorance
dist: 344.97101327

MSE: 253.68810876623377
detected
dist: 229.10696191953662
distIgnorance
dist: 2.23606797749979
MSE: 180.17126865671642
detected
dist: 4.242640687119285
MSE: 203.20260328239954
detected
dist: 231.65923249462776
distIgnorance
dist: 2.23606797749979
MSE: 214.01751279024006
detected
dist: 5.0
MSE: 117.99930106587455
detected
dist: 234.10467744152402
distIgnorance
dist: 1.0
MSE: 304.6227384868421
detected
dist: 5.0
MSE: 117.44971537001898
detected
dist: 236.75514777930383
distIgnorance
dist: 2.23606797749979
MSE: 272.69208333333336
detected
dist: 5.656854249492381
MSE: 164.3826024442847
detected
dist: 240.44126101815388
distIgnorance
dist: 1.4142135623730951
MSE: 190.1535116329637
detected
dist: 5.830951894845301
MSE: 1215.6234541577826
dist: 250.6671099286861
distIgnorance
616
dist: 125.53883861180172
MSE: 6905.662544931703
dist: 132.74034804836094
MSE: 7926.243169398907
617
dist: 244.38085031360376
distIgnorance
dist: 1.4142135623730951
MSE: 449.3613387978142
detected
dist: 5.656854249492

# Bonus Parts:
## 1.Color & 2.Speed insertion

In [15]:
def labellingColor(tube,color,percent = 0.15):
    
    index = 0
    for pack in tube:
        miniImage = pack[0]
        height,width,_ = miniImage.shape
        
        width = int(width * percent)
        height = int(height * percent)
        
        newImage = miniImage.copy()
        newImage[0:height,0:width] = color
        
        #cv2.rectangle(newImage,(0,0),(width,height),color,20)
        #newImage = newImage[0:height ,w//2 - width/2 : w//2 + width/2]
        
        tube[index] = (newImage,pack[1],pack[2])
        index += 1
        

def colorDist(mainPixel,backPixel):
    
    import math
    m_b,m_g,m_r = mainPixel
    
    # border(green) & time(red) ignorance
    if((m_b == 0 and m_g == 255 and m_r == 0)
               or
       (m_b == 0 and m_g == 0 and m_r == 255)):
        return 0
    
    b_b,b_g,b_r = backPixel
    dist = math.sqrt(pow((m_b - b_b),2) + 
                     pow((m_g - b_g),2) +
                     pow((m_r - b_r),2))
    
    return dist


def colorDetection(image,back,size):
    
    w = size[0]
    h = size[1]
    step = size[2]
    threshold = 25
    filteredPixels = []
    for i in range(0,w,step):
        for j in range(0,h,step):

            dist = colorDist(image[j,i],back[j,i])
            if(dist > threshold):
                filteredPixels.append(image[j,i])
    
    import numpy as np
    temp = np.array(filteredPixels)
    b,g,r = np.mean(temp,axis = 0)
    color = int(b),int(g),int(r)
    return color

def filterTubes(tubes,threshold):
    filteredTubes = []
    for tube in tubes:
        if(len(tube) >= threshold):
            filteredTubes.append(tube)
    return filteredTubes
       
    

def getCenter(pack):
    
    image,data,t = pack
    x,y,w,h = data
    centerY = y + h//2
    centerX = x + w//2
    return (centerX,centerY,t)

def timeDifference(start,end):
    global scale
    if( (end.minute >= start.minute) and (end.second >= start.second) ):
        minute = (end.minute - start.minute)*scale
        second = (end.second - start.second)*scale
        return (int(minute),int(second))
    else:
        minute = (end.minute - start.minute - 1)*scale
        second = (end.second - start.second + 60)*scale
        return (int(minute),int(second))

def calculateSpeed(firstCenter,lastCenter):
    
    deltaT = timeDifference(firstCenter[2],lastCenter[2])
    totTime = deltaT[0]*60 + deltaT[1]
    
    if(totTime == 0):
        totTime = 1
    
    import math
    dist = math.sqrt( pow((firstCenter[0] - lastCenter[0]),2) +
                      pow((firstCenter[1] - lastCenter[1]),2) )
    
    return dist/totTime
    
def speedingTube(tube):
    
    firstCenter = getCenter(tube[0])
    lastCenter = getCenter(tube[len(tube) - 1])
    speed = calculateSpeed(firstCenter,lastCenter)
    labellingSpeed(tube,speed)
    
def labellingSpeed(tube,speed):
    
    index = 0
    for pack in tube:
        
        miniImage = pack[0]
        height,width,_ = miniImage.shape
        
        newImage = miniImage.copy()
        
        cv2.putText(newImage, str(round(speed,1)),
                   (0,newImage.shape[0]//2), 
                    cv2.FONT_HERSHEY_SIMPLEX,0.35, (0, 0, 255), 1)
        
        tube[index] = (newImage,pack[1],pack[2])
        index += 1
            
# color Detection and Speed insertion :)
colors = []
# each tube's element -> image,data,t
step = 10
threshold = 10
newFilterTube = filterTubes(tubes,threshold)

cap = cv2.VideoCapture("Video1.avi")
fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
realDuration = frame_count/fps

#process time duration
startTimer = startTime.minute * 60 + startTime.second
endTimer =  endTime.minute * 60 + endTime.second
processDuration = endTimer - startTimer

scale = realDuration/processDuration


for tube in newFilterTube:
    index = len(tube)//2
    element = tube[index]
    miniImage,data,t = element
    # removing green parts
    h,w,_ = miniImage.shape
    x,y,w,h = data
    
    miniBackGround = background[y:y+h,x:x+w]
      
    color = colorDetection(miniImage,miniBackGround,(w,h,step))
    
    # percentage of color label -> optional
    # default value is 15 percent
    
    labellingColor(tube,color) 
    speedingTube(tube)
    colors.append(color)
    


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in ubyte_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in ubyte_scalars


# BriefVideoCreation :) & TimeInsertion

In [16]:
cap = cv2.VideoCapture("Video1.avi")
fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
realDuration = frame_count/fps

#process time duration
startTimer = startTime.minute * 60 + startTime.second
endTimer =  endTime.minute * 60 + endTime.second
processDuration = endTimer - startTimer

scale = realDuration/processDuration

def showTube(tube):
    for pack in tube:
        cv2.imshow("tube",pack[0])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
def timeDifference(start,end):
    global scale
    if( (end.minute >= start.minute) and (end.second >= start.second) ):
        minute = (end.minute - start.minute)*scale
        second = (end.second - start.second)*scale
        return (int(minute),int(second))
    else:
        minute = (end.minute - start.minute - 1)*scale
        second = (end.second - start.second + 60)*scale
        return (int(minute),int(second))

def filterTubes(tubes,threshold):
    filteredTubes = []
    for tube in tubes:
        if(len(tube) >= threshold):
            filteredTubes.append(tube)
    return filteredTubes
        

def insert(frame,someTubes):
    finished = []
    for i in range(len(someTubes)):
        if(len(someTubes[i]) == 0):
            finished.append(i)
    if(len(finished) > 0):
        return (False,finished)
    
    # all tubes has atLeast one pack
    # inserting intoPack
    
    for tube in someTubes:
        # tube is a Image,data,t
        # unpacking
        miniImage,data,t = tube[0]
        
        #adding time to miniImage

        difference = timeDifference(startTime,t)
        time = ('%02d:%02d'%(difference[0],difference[1]))
        cv2.putText(miniImage, time,
                   (2, miniImage.shape[0]- 2), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)
    
        # data is (x,y,w,h)
        x,y,w,h = data
        frame[y:y+h,x:x+w] = miniImage
        
            
    return (True,frame)
             
# brief video synapthis
threshold = 10
filteredTubes = filterTubes(tubes,threshold)

# initializing
noOfTubesPerFrame = 5
tubePointer = noOfTubesPerFrame
frameTubePack = filteredTubes[0:noOfTubesPerFrame]
    
briefFrames = []
    
#cv2.imshow("background",background)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
    
while(True):
    
    briefFrame = background.copy()
    done,result = insert(briefFrame,frameTubePack)

    if(not done):
        
        # tubes ended.
        if(tubePointer >= len(filteredTubes)):
            break
            
        # refilling finished Tubes.
        for index in result:
            if(not tubePointer >= len(filteredTubes)):
                frameTubePack[index] = filteredTubes[tubePointer]
                tubePointer += 1
    
    # done
    else:
        for tube in frameTubePack:
            # poping first pack
            tube.pop(0)
        briefFrames.append(result)

        
# creating video
height,width,layers=briefFrames[0].shape
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video = cv2.VideoWriter('output.avi',fourcc,20,(width,height))
print(len(briefFrames))

for frame in briefFrames:
    video.write(frame)
    
video.release()

519


# BonusParts
## 3.Direction (جهت یابی حرکت ها:مثلا شمالغرب به جنوبِ شرق)

In [17]:
# direction
def giveMeRegression(centers):
    x = []
    y = []
    
    for center in centers:
        x.append([center[0]])
        y.append([center[1]])
        
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(x,y)
    return model
    

def giveMeSlope(model,centers):
    
    firstDot = centers[0]
    secondDot = centers[1]
    
    first_reg = model.predict([[firstDot[0]]])[0][0]
    second_reg = model.predict([[secondDot[0]]])[0][0]
    
    deltaX = second_reg - first_reg
    deltaY = secondDot[0] - firstDot[0]
    if( deltaY == 0 ):
        deltaY = 0.001
        
    slope = deltaX/deltaY
    
    import math
    return int(math.degrees(math.atan((-1)*slope)))

def giveMeDirection(model,centers):
    
    slopeDegree = giveMeSlope(model,centers)
    
    firstDot = centers[0]
    secondDot = centers[ len(centers) - 1 ]
    
    
    if( -45 >= slopeDegree and slopeDegree >= -90):
        
        if((secondDot[1] > firstDot[1]) and
           (secondDot[0] > firstDot[0])):
            return "شمال غرب به سمت جنوب شرق"
        
        else:
            return "جنوب شرق به سمت شمال غرب"
        
    if( 0 >= slopeDegree and slopeDegree >= -45):
        
        if((secondDot[1] > firstDot[1]) and
           (secondDot[0] > firstDot[0])):
            return "غرب شمال به سمت شرق جنوب"
        else:
            return "شرق جنوب به سمت غرب شمال"
        
    if( 45 >= slopeDegree and slopeDegree >= 0):
        
        if((secondDot[1] < firstDot[1]) and
           (secondDot[0] > firstDot[0])):
            return "غرب جنوب به سمت شرق شمال"
        else:
            return "شرق شمال به سمت غرب جنوب"
        
        
    if( 90 >= slopeDegree and slopeDegree >= 45):
        
        if((secondDot[1] < firstDot[1]) and
           (secondDot[0] > firstDot[0])):
            return "جنوب غرب به سمت شمال شرق"
        else:
            return "شمال شرق به سمت جنوب غرب"

        
#processing (North East West South)
#filtering Tubes

threshold = 10
filteredTubes = filterTubes(mainTubes,threshold)

f = open("perTubeMoveMents.txt", "w",encoding="utf-8")

for tube in filteredTubes:
    centers = []
    for pack in tube:
        image,data,t = pack
        x,y,w,h = data
        centerY = y + h//2
        centerX = x + w//2
        centers.append((centerX,centerY))
        
    model = giveMeRegression(centers)
    string = giveMeDirection(model,centers)
    f.write(string + "\n")
    
f.close()

# clustering
#open and read the file after the appending:

perTubeMoveMents = open("perTubeMoveMents.txt", "r",encoding="utf-8")
clusteredMoveMents = open("clusteredMoveMents.txt", "w",encoding="utf-8")

  
NW_SE = "شمال غرب به سمت جنوب شرق"
SE_NW = "جنوب شرق به سمت شمال غرب"
WN_ES = "غرب شمال به سمت شرق جنوب"
ES_WN = "شرق جنوب به سمت غرب شمال"
WS_EN = "غرب جنوب به سمت شرق شمال"
EN_WS = "شرق شمال به سمت غرب جنوب"
SW_NE = "جنوب غرب به سمت شمال شرق"
NE_SW = "شمال شرق به سمت جنوب غرب"

directionDict = {
    
    NW_SE : 0,
    SE_NW : 0,
    WN_ES : 0,
    ES_WN : 0,
    WS_EN : 0,
    EN_WS : 0,
    SW_NE : 0,
    NE_SW : 0

} 

while(True):
    string = perTubeMoveMents.readline().strip()
    if(string != ""):
        directionDict[string] += 1
    else:
        break

# saving Result
for direct in directionDict:
    clusteredMoveMents.write("تعداد" + 
                             " " +
                             direct  +
                             " " +
                             "ها برابر است با" +
                             ": "    +
                             str(directionDict[direct]) +
                             "\n")

perTubeMoveMents.close()
clusteredMoveMents.close()

# BonusParts
## 4.GradientLine

In [18]:
# drawGradientLine

# background -> noOftubePerFrame -> draw , delay

def giveMeRegression(centers):
    x = []
    y = []
    
    for center in centers:
        x.append([center[0]])
        y.append([center[1]])
        
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(x,y)
    return model

def getFrame(backGround,centersPack):
    back = backGround.copy()
    
    for tube in centersPack:
        first,last = tube[0],tube[len(tube) - 1]
        model = giveMeRegression([first,last])
    
        line_thickness = 5
        b,g,r = 0,0,0
        step = 255//(len(tube) - 1)
    
        for i in range(len(tube) - 1):
            x1,y1 = tube[i]
            x2 = tube[i + 1][0]
            y2 = int(model.predict([[x2]])[0][0])
            b,g,r = step * i,step * i,step * i
            cv2.line(back, (x1, y1), (x2, y2), (b, g, r), thickness=line_thickness)
        
    return back
        
threshold = 10
filteredTubes = filterTubes(mainTubes,threshold)

packCenters = []
frames = []
for tube in filteredTubes:
    centers = []
    for pack in tube:
        image,data,t = pack
        x,y,w,h = data
        centerY = y + h//2
        centerX = x + w//2
        centers.append((centerX,centerY))
    
    packCenters.append(centers)

noOfGradientLinesPerFrame = 4

noOfFullFrame = len(packCenters)//noOfGradientLinesPerFrame
lastFrameLen = len(packCenters) - noOfFullFrame*noOfGradientLinesPerFrame

for i in range(noOfFullFrame):
    thisFrameCentersPack = packCenters[i*noOfGradientLinesPerFrame
                                      :
                                    (i+1)*noOfGradientLinesPerFrame]
    
    frame = getFrame(background,thisFrameCentersPack)
    frames.append(frame)
    
if(lastFrameLen > 0):
    lastCentersPack = packCenters[noOfFullFrame * noOfGradientLinesPerFrame:]
    frame = getFrame(background,lastCentersPack)
    frames.append(frame)

# frames Ready :)
    
# creating video
height,width,layers=frames[0].shape
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
video = cv2.VideoWriter('GradientLine.avi',fourcc,20,(width,height))
print(len(frames))

for frame in frames:
    video.write(frame)
    
video.release()

18
